In [1]:
%load_ext autoreload
%autoreload 2

from nli.data import Vocabulary

from datasets import load_from_disk
from tqdm import tqdm
import torch
import pickle

In [2]:
dataset_snli = load_from_disk('data/snli')


In [4]:
for split in dataset_snli:
    print(split)

test
train
validation


In [11]:
import torch
import pytorch_lightning as pl
import argparse

from nli.data import NLIDataModule, Vocabulary, DataSetPadding
from nli.models import AvgWordEmb, UniLSTM, BiLSTM, MaxPoolLSTM, MLP, NLINet
from nli.learner import Learner
import os

In [17]:
a = AvgWordEmb()
b = UniLSTM(2048)

In [24]:
a.__class__.__name__

'AvgWordEmb'

In [4]:
samples = [['obvious'], ['horrible'], ['crummy'], ['execrable', '.'], ['shallow', '.'], ['eh', '.'], ['abysmally', 'pathetic'], ['terrible', '.'], ['wishy-washy', '.'], ['spiderman', 'rocks'], ['delightfully', 'rendered'], ['surprisingly', 'insightful'], ['touchM-i', '!'], ['refreshing', '.'], ['fantastic', '!'], ['calculated', 'swill', '.'], ['thoroughly', 'awful', '.'], ['truly', 'terrible', '.'], ['by-the-numbers', 'yarn', '.'], ['amazingly', 'dopey', '.'], ['crikey', 'indeed', '.'], ['so-so', 'entertainment', '.'], ['cinematic', 'poo', '.'], ['extremely', 'bad', '.'], ['woefully', 'pretentious', '.'], ['warmed-over', 'hash', '.'], ['lacks', 'depth', '.'], ['under-rehearsed', 'and', 'lifeless'], ['unwieldy', 'contraption', '.'], ['feeble', 'comedy', '.'], ['disjointed', 'parody', '.'], ['two-bit', 'potboiler', '.'], ['painfully', 'padded', '.'], ['dramatically', 'lackluster', '.'], ['incoherence', 'reigns', '.'], ['mildly', 'amusing', '.'], ['fairly', 'run-of-the-mill', '.'], ['mildly', 'entertaining', '.'], ['insufferably', 'naive', '.'], ['amazingly', 'lame', '.'], ['predictably', 'melodramatic', '.'], ['rashomon-for-dipsticks', 'tale', '.'], ['no', 'surprises', '.'], ['thoroughly', 'enjoyable', '.'], ['family', 'fare', '.'], ['compellingly', 'watchable', '.'], ['often', 'hilarious', '.'], ['psychologically', 'savvy', '.'], ['one-of-a-kind', 'near-masterpiece', '.'], ['highly', 'engaging', '.'], ['beautifully', 'produced', '.'], ['psychologically', 'revealing', '.'], ['visually', 'captivating', '.'], ['morvern', 'rocks', '.'], ['genuinely', 'unnerving', '.'], ['deliciously', 'slow', '.'], ['exciting', 'documentary', '.'], ['a', 'muted', 'freak-out'], ['harmless', 'fun', '.'], ['oddly', 'compelling', '.'], ['delirious', 'fun', '.'], ['quietly', 'engaging', '.'], ['a', 'joyous', 'occasion'], ['everything', 'is', 'off', '.'], ['earnest', 'but', 'heavy-handed', '.'], ['one', 'lousy', 'movie', '.'], ['fluffy', 'and', 'disposible', '.'], ['aan', 'opportunity', 'wasted', '.'], ['storytelling', 'feels', 'slight', '.'], ['a', 'high-minded', 'snoozer', '.'], ['banal', 'and', 'predictable', '.'], ['brisk', 'hack', 'job', '.'], ['punitively', 'affirmational', 'parable', '.'], ['decent', 'but', 'dull', '.'], ['thin', 'period', 'piece', '.'], ['well-meant', 'but', 'unoriginal', '.'], ['odd', 'and', 'weird', '.'], ['draggin', "'", 'about', 'dragons'], ['a', 'dreary', 'movie', '.'], ['pompous', 'and', 'garbled', '.'], ['well-meaning', 'but', 'inert', '.'], ['shrewd', 'but', 'pointless', '.'], ['a', 'non-mystery', 'mystery', '.'], ['what', 'an', 'embarrassment', '.'], ['a', 'noble', 'failure', '.'], ['a', 'relative', 'letdown', '.'], ['a', 'puzzling', 'experience', '.'], ['just', 'not', 'campy', 'enough'], ['aggravating', 'and', 'tedious', '.'], ['an', 'awful', 'snooze', '.'], ['just', 'plain', 'silly', '.'], ['a', 'less-than-thrilling', 'thriller', '.'], ['black-and-white', 'and', 'unrealistic', '.'], ['anemic', ',', 'pretentious', '.'], ['i', 'hate', 'this', 'movie'], ['grating', 'and', 'tedious', '.'], ['it', 'bites', 'hard', '.'], ['a', 'pretentious', 'mess', '.'], ['predictably', 'soulless', 'techno-tripe', '.'], ['arty', 'gay', 'film', '.'], ['a', 'half-assed', 'film', '.'], ['bland', 'but', 'harmless', '.'], ['a', 'dreary', 'indulgence', '.'], ['tends', 'to', 'plod', '.'], ['a', 'well-crafted', 'letdown', '.'], ['boring', 'and', 'meandering', '.'], ['less', 'than', 'fresh', '.'], ['a', 'lame', 'comedy', '.'], ['a', 'reality-snubbing', 'hodgepodge', '.'], ['degenerates', 'into', 'hogwash', '.'], ['meandering', 'and', 'confusing', '.'], ['an', 'opportunity', 'missed', '.'], ['inconsequential', 'road-and-buddy', 'pic', '.'], ['a', 'movie', 'to', 'forget'], ['more', 'precious', 'than', 'perspicacious'], ['an', 'intriguing', 'near-miss', '.'], ['bearable', '.', 'barely', '.'], ['staggeringly', 'dreadful', 'romance', '.'], ['well-made', 'but', 'mush-hearted', '.'], ['a', 'real', 'snooze', '.'], ['effective', 'but', 'too-tepid', 'biopic'], ['an', 'imaginative', 'comedy/thriller', '.'], ['an', 'exhilarating', 'experience', '.'], ['troubling', 'and', 'powerful', '.'], ['tailored', 'to', 'entertain', '!'], ['a', 'modest', 'masterpiece', '.'], ['never', 'once', 'predictable', '.'], ['warm', 'and', 'exotic', '.']]

In [7]:
dataset_snli = load_from_disk('data/snli')
samples = dataset_snli['train']['premise'] + dataset_snli['train']['hypothesis']
vocab = Vocabulary(samples, path_to_vec = "data/glove.840B.300d.txt")

Creating dictionary: 100%|██████████| 1098734/1098734 [00:02<00:00, 457523.47it/s]
Creating word vectors from data/glove.840B.300d.txt:   0%|          | 1000/2196017 [00:00<11:35, 3157.46it/s]

Words in sample: 36549
Words in wordvec: 779
Overlapping words: 779


In [2]:
with open('store/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

NameError: name 'vocab' is not defined

In [7]:
vocab = pickle.load(open('store/vocab.pkl', 'rb'))


EOFError: Ran out of input

In [5]:
vocab = pickle.load(open('store/vocab.pkl', 'rb'))

hidden_dim = 2048

encoder = UniLSTM(hidden_dim)
classifier = MLP(hidden_dim*4)

In [6]:
net = NLINet(encoder, classifier, vocab)
model = Learner(net)

In [7]:
ckpt_path = 'lightning_logs/version_0/checkpoints/epoch=2-step=48.ckpt'

In [8]:
model = Learner.load_from_checkpoint(ckpt_path, net=net)
model.eval();

In [13]:
prep_sent = lambda sent: DataSetPadding(None, vocab).prepare_sent(sent)

In [33]:
sent_ids, slens = zip(*[prep_sent(sent) for sent in samples])
sent_ids = torch.stack(sent_ids)
slens    = torch.tensor(slens)

In [37]:
embedding = model.net.encode(sent_ids, slens)

In [32]:
argsversion = 'test'

ckpt_path = 'avg_word_emb'

versions = os.listdir(os.path.join('logs', ckpt_path))
version = versions[0] if len(versions) == 1 else argsversion
ckpts = os.listdir(os.path.join('logs', ckpt_path, version, 'checkpoints'))
assert len(ckpts) == 1
ckpt_path = os.path.join('logs', ckpt_path, version, 'checkpoints', ckpts[0])

ckpt_path

'logs/avg_word_emb/test/checkpoints/epoch=3-step=64.ckpt'